# routing outputs to a specific cell


# THIS IS A WORK IN PROGRESS!


## About this notebook

This notebook belongs to a series of small projects which aim is to evaluate the [Jupyter](http://jupyter.org/) ecosystem for science experiments control. The main idea is use the _Juypter notebook_ as a convergence platform in order to offer a fully featured environment to scientists. 

## Topic of the day

When working in the notebook, we might have some asynchronous activity running in several cells. We could, for instance, have a couple of live plots updated regularly in their associated cell. In such a case, we certainly want any output generated by a given asynchronous process to be routed to the cell to which it's attached. However, so far, the notebook routes the outputs to the _current cell_ - i.e. to the one in which we are currently working. It means that any output produce from the python backend - e.g. messages or even some widgets - will drop into the _current cell_. That could be quite annoying. 

The remaining content details a solution based on [this expert trick](https://nbviewer.jupyter.org/gist/minrk/049545c1edcf20415bb3d68f16047628). A big thanks to [@minrk](https://github.com/minrk) for sharing his expertise.

In [ ]:
import sys
from contextlib import wraps, contextmanager
from IPython import get_ipython

class CellContext(object):
    
    def __init__(self):
        k = get_ipython().kernel
        self._ident = k._parent_ident
        self._header = k._parent_header
        self._save_publish_status = k._publish_status
        k._publish_status = self.__disable_once
       
    def __disable_once(self, *args, **kw):
        get_ipython().kernel._publish_status = self._save_publish_status
        
    def __call__(self):
        return (self._ident, self._header)
        
@contextmanager
def cell_context(context):
    try:
        assert(isinstance(context, CellContext))
        kernel = get_ipython().kernel
        save_context = (kernel._parent_ident, kernel._parent_header)
        sys.stdout.flush()
        sys.stderr.flush()
        kernel.set_parent(*context())
    except Exception as e:
        print(e)
        return
    try:
        yield
    except:
        raise
    finally:
        sys.stdout.flush()
        sys.stderr.flush()
        kernel.set_parent(*save_context)

In [ ]:
import logging

class NotebookCellContent(object):

    def __init__(self):
        self._context = CellContext()
        self._logger = logging.getLogger('jupyter.for.controls')
        try:
            #TODO: understand why we need this workaround!
            s = self._logger.handlers[0].stream
        except IndexError:
            handler = logging.StreamHandler(sys.stdout)
            handler.setFormatter(logging.Formatter(logging.BASIC_FORMAT, None))
            self._logger.addHandler(handler)

    @property
    def context(self):
        return self._cell_context

    def display(self, widgets_layout):
        with cell_context(self._context):
            display(widgets_layout)
       
    def clear_output(self):
        with cell_context(self._context):
            clear_output()
         
    def reset_output(self):
        with cell_context(self._context):
            reset_output()
            
    @property
    def logger(self):
        return self._logger

    def set_logging_level(self, level):
        self._logger.setLevel(level)

    def print(self, *args):
        with cell_context(self._context):
            print(*args)
            
    def debug(self, msg, *args, **kwargs):
        with cell_context(self._context):
            self._logger.debug(msg, *args, **kwargs)

    def info(self, msg, *args, **kwargs):
        with cell_context(self._context):
            self._logger.info(msg, *args, **kwargs)

    def warning(self, msg, *args, **kwargs):
        with cell_context(self._context):
            self._logger.warning(msg, *args, **kwargs)

    def error(self, msg, *args, **kwargs):
        with cell_context(self._context):
            self._logger.error(msg, *args, **kwargs)

    def critical(self, msg, *args, **kwargs):
        with cell_context(self._context):
            self._logger.critical(msg, *args, **kwargs)

    def exception(self, msg, *args, **kwargs):
        with cell_context(self._context):
            self._logger.exception(msg, *args, **kwargs)

In [ ]:
ncc = NotebookCellContent()
ncc.set_logging_level(logging.DEBUG)

In [ ]:
ncc.print("foo")
ncc.debug("foo")
print('bar')

### Reusing some code from a previous topic 
Here we use a modified version of some code we wrote for the previous topic we treated in [01_about_sessions_life_cycle](https://github.com/nleclercq/jupyter-for-controls/blob/master/01_about_sessions_life_cycle.ipynb). 
We reuse it cause it provides the asynchronous activity we need here. Simply execute the cell and see you in the cell one...

In [ ]:
import socket
import numpy as np
from collections import deque
from IPython.display import HTML, clear_output
from tornado.ioloop import IOLoop
from bokeh.server.server import Server
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler
from bokeh.embed import autoload_server
from bokeh.io import reset_output
from bokeh.io import output_notebook
from bokeh.plotting import figure
from bokeh.plotting.figure import Figure
from bokeh.models.glyphs import Rect
from bokeh.models import ColumnDataSource
from bokeh.models.widgets import Slider, Button
from bokeh.layouts import layout, widgetbox

output_notebook()

In [ ]:
class BokehSession(NotebookCellContent):
    
    def __init__(self):
        NotebookCellContent.__init__(self)
        """the associated bokeh document (for experts only)"""
        self._doc = None
        """periodic callback period in seconds - defaults to None (i.e. disabled)"""
        self._callback_period = None
        
    def open(self):
        """open the session"""
        BokehServer.open_session(self)
        
    def close(self):
        """close the session""" 
        if self._doc:
            self._doc.add_next_tick_callback(self.__cleanup)
      
    def __cleanup(self):
        """cleanup the session""" 
        try:
            # clear document content (i.e. remove roots)
            self._doc.clear()
        except Exception as e:
            self.exception(e)
        try:
            # reset_output
            self.reset_output()
        except Exception as e:
            self.exception(e)
        try:  
            # close the session (will remove all callbacks)
            BokehServer.close_session(self)
        except Exception as e:
            self.exception(e) 
        # finally clear cell outputs - e.g. logging (this is an ipython call - not a bokeh one)
        self.clear_output()
        
    def setup_model(self):
        """return the session model or None if no model"""
        return None

    def periodic_callback(self):
        """periodic callback (default impl. does nothing)"""
        pass
    
    @property 
    def callback_period(self):
        """return the (periodic) callback period in seconds or None (i.e. disabled)"""
        return self._callback_period

    @callback_period.setter 
    def callback_period(self, p):
        """set the (periodic) callback period in seconds or None to disable the callback"""
        self._callback_period = p
        if self._doc is not None:
            BokehServer.update_callback_period(self)

In [ ]:
class BokehServer(object):

    __bkh_app__ = None
    __bkh_srv__ = None
    __srv_url__ = None
    __sessions__ = deque()
        
    @staticmethod
    def __start_server():
        app = Application(FunctionHandler(BokehServer.__entry_point))
        srv = Server(
            {'/': app},
            io_loop=IOLoop.instance(),
            port=0,
            host='*',
            allow_websocket_origin=['*']
        )
        srv.start()
        srv_addr = srv.address if srv.address else socket.gethostbyname(socket.gethostname())
        BokehServer.__bkh_srv__ = srv
        BokehServer.__bkh_app__ = app
        BokehServer.__srv_url__ = 'http://{}:{}'.format(srv_addr, srv.port)
        
    @staticmethod
    def __entry_point(doc):
        try:
            session = BokehServer.__sessions__.pop() #TODO: should we lock BokehServer.__sessions__? 
            session._doc = doc
            model = session.setup_model()
            if model:
                doc.add_root(model)
            BokehServer.__add_periodic_callback(session)
        except Exception as e:
            print(e)
        
    @staticmethod
    def __add_periodic_callback(session):
        assert(isinstance(session, BokehSession))
        pcb = session.periodic_callback
        try:
            session._doc.remove_periodic_callback(pcb)
        except:
            pass
        prd = session.callback_period
        if prd is not None:
            session._doc.add_periodic_callback(pcb, max(250, 1000. * prd))
        
    @staticmethod
    def open_session(new_session):
        assert(isinstance(new_session, BokehSession))
        if not BokehServer.__bkh_srv__:
            BokehServer.__start_server()
        BokehServer.__sessions__.appendleft(new_session) #TODO: should we lock BokehServer.__sessions__? 
        script = autoload_server(model=None, url=BokehServer.__srv_url__)
        html_display = HTML(script)
        display(html_display)
        
    @staticmethod
    def close_session(session):
        """totally experimental attempt to destroy a session from python!"""
        assert(isinstance(session, BokehSession))
        session_id = session._doc.session_context.id
        session = BokehServer.__bkh_srv__.get_session('/', session_id)
        session.destroy()
        
    @staticmethod
    def update_callback_period(session):
        assert(isinstance(session, BokehSession))
        BokehServer.__add_periodic_callback(session)
        
    @staticmethod
    def print_info(called_from_session_handler=False):
        if not BokehServer.__bkh_srv__:
            print("no Bokeh server running") 
            return
        try:
            print("Bokeh server URL: {}".format(BokehServer.__srv_url__))
            sessions = BokehServer.__bkh_srv__.get_sessions()
            num_sessions = len(sessions)
            if called_from_session_handler:
                num_sessions += 1
            print("Number of opened sessions: {}".format(num_sessions))
        except Exception as e:
            print(e)

In [ ]:
class LivePlot(BokehSession):
    
    def __init__(self):
        BokehSession.__init__(self)
        self.callback_period = 1.
        self._np = 100
        self._widgets_layout = None
        columns = dict()
        columns['x'] = self._gen_x_scale()
        columns['y'] = self._gen_y_random_data()
        self._cds = ColumnDataSource(data=columns)

    def _gen_x_scale(self):
        """x data"""
        return np.linspace(1, self._np, num=self._np, endpoint=True)
    
    def _gen_y_random_data(self):
        """y data"""
        return np.random.rand(self._np)
    
    def __on_update_period_change(self, attr, old, new):
        """called when the user changes the refresh period using the dedicated slider"""
        self.callback_period = new
        
    def __on_num_points_change(self, attr, old, new):
        """called when the user changes the number of points using the dedicated slider"""
        self._np = int(new)
        
    def setup_model(self):
        """setup the session model then return it"""
        # a slider to control the update period
        rrs = Slider(start=0.25, end=2, step=0.25, value=self.callback_period, title="Updt.period [s]",)
        rrs.on_change("value", self.__on_update_period_change)
        # a slider to control the number of points
        nps = Slider(start=0, end=1000, step=10, value=self._np, title="Num.points")
        nps.on_change("value", self.__on_num_points_change)
        # a button to 'close' the session
        cbt = Button(label='close')
        cbt.on_click(self.close)
        # the figure and its content
        p = figure(plot_width=650, plot_height=200)
        p.toolbar_location = 'above'
        p.line(x='x', y='y', source=self._cds, color="navy", alpha=0.5)
        # widgets are placed into a dedicated layout
        self._widgets_layout = widgetbox(nps, rrs, cbt)
        # arrange all items into a layout then return it as the session model
        return layout([[self._widgets_layout, p]])
    
    def periodic_callback(self):
        """periodic activity"""
        self.debug('MyExtendedSession.periodic_callback <<')
        self._cds.data.update(x=self._gen_x_scale(), y=self._gen_y_random_data())
        self.debug('MyExtendedSession.periodic_callback >>')

### Let's create some asynchronous activity 

In [ ]:
LivePlot().open()

In [ ]:
print('foo')